# Phylogenetic Tree

# Creating a distance tree
We will use ```phylip``` to create a distance tree from the p53 protein alignment that we created eariler using ```muscle```. We can use a script to convert an existing aligment into ```phylip``` format (I wrote such a script called ```fasta2phylip.pl```) or we can realign the sequences using ```muscle```, but this time we can directly output in ```phylip``` format. We will use the following program from the ```phylip``` package:

1. ```protdist``` creates a distance matrix from an alignment
2.	```neighbor``` will create a neighbor-joining tree

Remember, all programs in phylip will take an ```infile``` and will create an ```outfile```.


## Creating the alignment 
Copy the fasta file containing the p53 sequences into current directory. ```muscle``` can output two different ```phylip``` formats. We will use "interleaved". Note, we could have output several formats from ```muscle``` simultaneously during alignment, like this:

In [ ]:
muscle -in p53.fas -fastaout p53.afa -phyiout p53.phy 

## Get ```phylip```

In [ ]:
wget http://evolution.gs.washington.edu/phylip/download/phylip-3.697.tar.gz
tar -xvzf phylip-3.697.tar.gz
cd phylip-3.697/src
#pwd
make -f Makefile.unx install
cd ../exe
export PATH=$PATH:`pwd`
cd ../..
pwd
which protdist

## Running ```protdist```

In [ ]:
#cp p53.phy infile
protdist <<EOF
p53.phy
y
EOF


This will create the distance matrix in a file ```outfile```. We need to rename this file so that it can be read my `neighbor`.

In [ ]:
#rm infile
mv outfile infile



## Running ```neighbor```

In [ ]:
neighbor <<EOF
y
EOF

The tree will be produced in a file called ```outtree```:

In [ ]:
cat outtree

This is a newick format tree.

## Viewing the tree
To view the tree we can use ```FigTree```. You can download it from here:

http://tree.bio.ed.ac.uk/software/figtree/

Download and run the software as

`java -jar figtree.jar`

![Tree generated using ```neighbor```](figures/outtree.pdf)

We can also use R package ape to view the file. 

## Bootstrapping

To get the statistical measurement of tree, we can use "bootstrapping". We have to start with the raw ```phylip``` file.

In [ ]:
cd phylip-3.697/exe
export PATH=$PATH:`pwd`
cd ../..
#cp p53.phy infile
rm infile
cp outtree njtree.tre
cp p53.phy infile
seqboot<<EOF
y
3
EOF
rm infile
mv outfile infile

Now follow the exact step described earlier. Don't forget to change the option **"Multple data sets"**. The treefile will now have 100 trees (if you have used 100 bootstrap datasets). 

In [ ]:
protdist <<EOF
M
D
100
y
EOF
rm infile
mv outfile infile

## Run neighbor again

In [ ]:
rm outtree
neighbor <<EOF
M
100
3
y
EOF

After you have created 100 trees. Run ```consense``` to get a consensus tree:

In [ ]:
rm outfile
mv outtree intree
consense <<EOF
y
EOF
mv outtree neighbor_bootstrap.tre



# Maximum likelihood tree using RAxML

RAxML is a very efficient maximum likelihood phylogenetic analysis software that can run on a HPC settings. The software is generally used for very large datasets.

Download RAxML from https://github.com/stamatak/standard-RAxML. There are several way to compile RAxML. For use with our virtual machine we will use a very simple option.

In [ ]:
git clone https://github.com/stamatak/standard-RAxML.git
cd standard-RAxML/
make -f Makefile.gcc
rm *.o
cd ..

RAxML requires specification of a model. For nucleotide sequence the best option GTRGAMMA. For protein sequence any of the JTT models are fine. They are expanded PAM matrices.

But we can tell RAXML for find the best model for us.

In [ ]:
#cd 05-MolEvol
./standard-RAxML/raxmlHPC -m PROTGAMMAAUTO -s p53.afa -p 12345 -n p53

This will create the tree as ```RAxML_bestTree.p53```. 

We can also run a bootstrap analysis:

In [ ]:
cat RAxML_bestTree.p53

In [ ]:
standard-RAxML/raxmlHPC -m PROTGAMMAJTT -s p53.afa -p 12345 -# 10 -f a -n p53_c -x 12345

Unlike ```consense``` in ```phylip```. RAxML already geneartes a tree will bootstrap values already included. This is the file ```RAxML_bipartitions.p53_c``` How nice!














